In [1]:
!git clone https://github.com/openai/shap-e.git  > /dev/null 2>&1
%cd shap-e/
!pip install -e . > /dev/null 2>&1
!conda install -y conda-forge numpy-stl > /dev/null 2>&1

/home/fish/Documents/AI_Computer_Vision/Shap-E/shap-e
^C
^C


In [ ]:
import torch
from stl import mesh
import os
import glob

from shap_e.models.download import load_model
from shap_e.util.data_util import load_or_create_multimodal_batch
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
xm = load_model('transmitter', device=device)

model_path = "example_data/cactus/object.obj"

batch = load_or_create_multimodal_batch(
    device,
    model_path=model_path,
    mv_light_mode="basic",
    mv_image_size=256,
    cache_dir="example_data/cactus/cached",
    verbose=True, 
)
with torch.no_grad():
    latent = xm.encoder.encode_to_bottleneck(batch)

    render_mode = 'stf' 
    size = 128 

    cameras = create_pan_cameras(size, device)
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

In [ ]:
def create_input():
    folder_name = "Input"
    if os.path.exists(folder_name):
        print(f"The folder '{folder_name}' already exists.")
    else:
        os.makedirs(folder_name)
        print(f"The folder '{folder_name}' has been created.")
        
def get_all_obj():
    folder_path = "things"
    obj_files = glob.glob(f"{folder_path}/**/*.obj", recursive=True)
    file_names = [os.path.basename(file) for file in obj_files]
    return obj_files, file_names

def stl_to_obj(Path,Name):
  stl_mesh = mesh.Mesh.from_file(f'{Name}.stl')
  stl_mesh.save(f'Input/{Name}.obj', mode=stl.Mode.ASCII)
    
def finetune_latent(path,name):
  model_path = f"{path}/{name}.obj"
  batch = load_or_create_multimodal_batch(
      device,
      model_path=model_path,
      mv_light_mode="basic",
      mv_image_size=256,
      cache_dir="example_data/cactus/cached",
      verbose=True, # this will show Blender output during renders
  )
  with torch.no_grad():
      latent = xm.encoder.encode_to_bottleneck(batch)

In [ ]:
obj_files, file_names = get_all_obj()
for path, name in zip(obj_files, file_names):
    stl_to_obj(path,name)